**Cài đặt thư viện UnderTheSea để hỗ trợ tách từ trong tiếng Việt. Xem thêm các tính năng của thư viện tại: https://github.com/undertheseanlp/underthesea**

In [1]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.0 MB 7.9 MB/s 
     |████████████████████████████████| 1.0 MB 42.9 MB/s 
     |████████████████████████████████| 591 kB 23.9 MB/s 


**Import các thư viện cần thiết**

In [2]:
import os
import re
import math
from underthesea import text_normalize
from underthesea import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance

**Lấy dữ liệu các tài liệu/văn bản mẫu từ**: https://github.com/HUTECH-OpenCourseWare/IRS.git

In [3]:
!git clone https://github.com/HUTECH-OpenCourseWare/IRS.git

Cloning into 'IRS'...
remote: Enumerating objects: 271, done.
remote: Counting objects: 100% (271/271), done.
remote: Compressing objects: 100% (271/271), done.
remote: Total 271 (delta 0), reused 271 (delta 0), pack-reused 0
Receiving objects: 100% (271/271), 441.48 KiB | 1.79 MiB/s, done.


**Tiến hành thử nghiệm với danh sách các tài liệu/văn bản thuộc các chủ đề khác nhau**

In [4]:
# Chọn danh sách các chủ đề của tài liệu/văn bản cho thử nghiệm
topics = [
    'the-thao',
    'giao-duc',
    'khoa-hoc'
]

# Tạo một tập dữ liệu thử nghiệm gồm các tài liệu/văn bản thuộc về 2-3 chủ đề
# Cấu trúc dữ liệu dạng list - lưu thông tin danh sách các tài liệu/văn bản thuộc chủ đề khác nhau
# Mỗi tài liệu/văn bản sẽ tổ chức dạng 1 tuple với: (topic, nội_dung_văn_bản, danh_sách_token)
D = []

**Tiến hành viết một số hàm hỗ trợ cho việc đọc dữ liệu, xử lý và tách từ trong tiếng Việt.**

In [5]:
# Viết hàm tiền xử lý và tách từ tiếng Việt
def preprocess(doc):
  # Tiến hành xử lý các lỗi từ/câu, dấu câu, v.v. trong tiếng Việt với hàm text_normalize
  normalized_doc = text_normalize(doc)
  # Tiến hành tách từ
  tokens = word_tokenize(normalized_doc)
  # Tiến hành kết hợp các từ ghép trong tiếng Việt bằng '_'
  combined_tokens = [token.replace(' ', '_') for token in tokens]
  return (normalized_doc, combined_tokens)

# Viết hàm lấy danh sách các văn bản/tài liệu thuộc các chủ đề khác nhau
def fetch_doc_by_topic(topic):
  data_root_dir_path = '/content/IRS/data/vnexpress/{}'.format(topic)
  docs = []
  for file_name in os.listdir(data_root_dir_path):
    file_path = os.path.join(data_root_dir_path, file_name)
    with open(file_path, 'r', encoding='utf-8') as f:
      lines = []
      for line in f:
        line = line.lower().strip()
        lines.append(line)
    doc = " ".join(lines)
    clean_doc = re.sub('\W+',' ', doc)
    (normalized_doc, tokens) = preprocess(clean_doc)
    docs.append((topic, normalized_doc, tokens))
  return docs

**Tiến hành tạo tập dữ liệu thử nghiệm với các tài liệu/văn bản thuộc danh sách chủ đề [topics] đã lựa chọn bên trên**

In [6]:
# Cấu trúc dữ liệu dictionary để lưu thông tin chủ đề-tài liệu, nhằm hỗ trợ cho việc tìm kiếm nhanh
topic_doc_idxes_dict = {}
doc_idx_topic_dict = {}

# Duyệt qua từng chủ đề
doc_idx = 0
for topic in topics:
  current_topic_docs = fetch_doc_by_topic(topic)
  topic_doc_idxes_dict[topic] = []
  for (topic, normalized_doc, tokens) in current_topic_docs:
    topic_doc_idxes_dict[topic].append(doc_idx)
    doc_idx_topic_dict[doc_idx] = topic
    doc_idx+=1
  D += current_topic_docs

doc_size = len(D)

print('Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [{}]'.format(doc_size))
for topic in topic_doc_idxes_dict.keys():
  print(' - Chủ đề [{}] có [{}] tài liệu/văn bản.'.format(topic, len(topic_doc_idxes_dict[topic])))

Hoàn tất, tổng số lượng tài liệu/văn bản đã lấy: [109]
 - Chủ đề [the-thao] có [39] tài liệu/văn bản.
 - Chủ đề [giao-duc] có [35] tài liệu/văn bản.
 - Chủ đề [khoa-hoc] có [35] tài liệu/văn bản.


**Tiến hành biến đổi các tài liệu/văn bản trong tập (D) về dạng các TF-IDF vectors - trong bài thực hành này chúng ta sẽ sử dụng thư viện Scikit-Learn (TfidfVectorizer) https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html**

In [7]:
# Khởi tạo đối tượng TfidfVectorizer
vectorizer = TfidfVectorizer()

# Chúng ta sẽ tạo ra một tập danh sách các tài liệu/văn bản dạng list đơn giản để thư viện Scikit-Learn có thể đọc được
sk_docs = []

# Duyệt qua từng tài liệu/văn bản có trong (D)
for (topic, normalized_doc, tokens) in D:
  # Chúng ta sẽ nối các từ/tokens đã được tách để làm thành một văn bản hoàn chỉnh
  text = ' '.join(tokens)
  sk_docs.append(text)

# Tiến hành chuyển đổi các tài liệu/văn bản về dạng các TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(sk_docs)

# Chuyển ma trận tfidf_matrix từ dạng cấu trúc thưa sang dạng đầy đủ để thuận tiện cho việc tính toán
tfidf_matrix = tfidf_matrix.todense()

**Tiến hành thử nghiệm truy vấn tìm kiếm một số tài liệu/văn bản dựa trên truy vấn (q)**

In [8]:
# Viết hàm giúp chuyển đổi truy vấn dạng text sang tfidf vector
def parse_query(query_text):
  (normalized_doc, combined_tokens) = preprocess(query_text)
  query_text = ' '.join(combined_tokens)
  query_tfidf_vector = vectorizer.transform([query_text])[0].todense()
  return query_tfidf_vector

# Viết hàm giúp tìm kiếm top-k (mặc định 10) các kết quả tài liệu/văn bản tương đồng với truy vấn 
def search(query_tfidf_vector, top_k = 10):
  search_results = {}
  for doc_idx, doc_tfidf_vector in enumerate(tfidf_matrix):
      # Tính mức độ tương đồng giữa truy vấn (q) và từng tài liệu/văn bản (doc_idx) bằng độ đo cosine
      cs_score = 1 - distance.cosine(query_tfidf_vector, doc_tfidf_vector)
      search_results[doc_idx] = cs_score
  # Tiến hành sắp xếp các tài liệu/văn bản theo mức độ tương đồng từ cao -> thấp
  sorted_search_results = sorted(search_results.items(), key=lambda item: item[1], reverse=True)
  print('Top-[{}] tài liệu/văn bản có liên quan đến truy vấn.'.format(top_k))
  for idx, (doc_idx, sim_score) in enumerate(sorted_search_results[:top_k]):
    print(' - [{}]. Tài liệu [{}], chủ đề: [{}] -> mức độ tương đồng: [{:.6f}]'.format(idx + 1, doc_idx, doc_idx_topic_dict[doc_idx], sim_score))

In [9]:
# Thử một truy vấn với chủ đề [the-thao]
query_text = 'bóng đá'

# Chuyển đổi truy vấn về dạng vector tfidf
query_tfidf_vector = parse_query(query_text)

# Chúng ta sẽ thử tìm kiếm với top-10 kết quả
top_k = 10

# Tiến hành tìm kiếm thử trong tập dữ liệu với truy vấn
search(query_tfidf_vector, top_k)

Top-[10] tài liệu/văn bản có liên quan đến truy vấn.
 - [1]. Tài liệu [14], chủ đề: [the-thao] -> mức độ tương đồng: [0.138164]
 - [2]. Tài liệu [35], chủ đề: [the-thao] -> mức độ tương đồng: [0.084354]
 - [3]. Tài liệu [103], chủ đề: [khoa-hoc] -> mức độ tương đồng: [0.072400]
 - [4]. Tài liệu [34], chủ đề: [the-thao] -> mức độ tương đồng: [0.029059]
 - [5]. Tài liệu [107], chủ đề: [khoa-hoc] -> mức độ tương đồng: [0.027820]
 - [6]. Tài liệu [12], chủ đề: [the-thao] -> mức độ tương đồng: [0.027710]
 - [7]. Tài liệu [21], chủ đề: [the-thao] -> mức độ tương đồng: [0.026200]
 - [8]. Tài liệu [18], chủ đề: [the-thao] -> mức độ tương đồng: [0.025886]
 - [9]. Tài liệu [26], chủ đề: [the-thao] -> mức độ tương đồng: [0.025115]
 - [10]. Tài liệu [33], chủ đề: [the-thao] -> mức độ tương đồng: [0.024014]


**Chúng ta có thể thấy kết quả trên có một số tài liệu có nhãn chủ đề là [khoa-hoc] (tài liệu có mã: 108, 101) có thể không liên quan đến truy vấn - do đó, đóng vai trò là người dùng, chúng ta sẽ:**

*   Chọn tất cả các tài liệu/văn bản có nhãn chủ đề là [khoa-hoc], vào tập: $D_{irel}$
*   Và chọn tất cả các tài liệu/văn bản có nhãn chủ đề là [the-thao], vào tập: $D_{rel}$

In [10]:
# Xác định danh sách các tài liệu/văn bản có liên quan (D_rel) và không liên quan (D_irel) đến truy vấn (q)
D_rel = [27, 32, 0, 33, 24, 19, 7, 5]
D_irel = [108, 101]

Tiến hành mở rộng/tái cấu trúc lại truy vấn với phép công và trừ vectors. Cụ thể:

*   Với tập $D_{rel}$ ($d^{rel} \in D_{rel}$): $$\vec{q}^{QR}=\vec{q}+\vec{d}^{rel}$$
*   Với tập $D_{irel}$ ($d^{irel} \in D_{irel}$): $$\vec{q}^{QR}=\vec{q}\vec{d}^{irel}$$



In [11]:
# Tạo mới một query mở rộng [query_tfidf_vector_QR] cho [query_tfidf_vector]
query_tfidf_vector_QR = query_tfidf_vector

# Với tập (D_rel) tiến hành cộng các tfidf vector của các tài liệu/văn bản trong đó với query_tfidf_vector_QR
for doc_idx in D_rel:
  doc_tfidf_vector_rel = tfidf_matrix[doc_idx]
  query_tfidf_vector_QR += doc_tfidf_vector_rel

# Với tập (D_irel) tiến hành trừ các tfidf vector của các tài liệu/văn bản trong đó với query_tfidf_vector_QR
for doc_idx in D_irel:
  doc_tfidf_vector_irel = tfidf_matrix[doc_idx]
  query_tfidf_vector_QR -= doc_tfidf_vector_irel

**Chúng ta tiến hành thử truy vấn lại các văn bản/tài liệu với truy vấn đã được mở rộng/tái cấu trúc [query_tfidf_vector_QR]**

In [12]:
# Tiến hành tìm kiếm lại với truy vấn mới[query_tfidf_vector_QR]
search(query_tfidf_vector_QR, top_k)

Top-[10] tài liệu/văn bản có liên quan đến truy vấn.
 - [1]. Tài liệu [33], chủ đề: [the-thao] -> mức độ tương đồng: [0.472643]
 - [2]. Tài liệu [5], chủ đề: [the-thao] -> mức độ tương đồng: [0.465699]
 - [3]. Tài liệu [7], chủ đề: [the-thao] -> mức độ tương đồng: [0.444015]
 - [4]. Tài liệu [0], chủ đề: [the-thao] -> mức độ tương đồng: [0.440115]
 - [5]. Tài liệu [32], chủ đề: [the-thao] -> mức độ tương đồng: [0.416658]
 - [6]. Tài liệu [19], chủ đề: [the-thao] -> mức độ tương đồng: [0.380849]
 - [7]. Tài liệu [29], chủ đề: [the-thao] -> mức độ tương đồng: [0.372405]
 - [8]. Tài liệu [27], chủ đề: [the-thao] -> mức độ tương đồng: [0.364751]
 - [9]. Tài liệu [24], chủ đề: [the-thao] -> mức độ tương đồng: [0.364675]
 - [10]. Tài liệu [9], chủ đề: [the-thao] -> mức độ tương đồng: [0.342513]


**Chúng ta có thể thấy các tài liệu với chủ đề [the-thao] đã được đẩy về lên trên trong danh sách top kết quả tìm kiếm và các tài liệu/văn bản [khoa-hoc] đã bị đẩy xuống dưới - không nằm trong top các kết quả nữa**